In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.3 MB/s eta 0:00:00


In [ ]:
!pip install torchvision 
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", forced_bos_token_id=0).to(device)

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("ccdv/lsg-bart-large-4096", 
    trust_remote_code=True, 
    num_global_tokens=16,
    block_size=64,
    sparse_block_size=64,
    sparsity_factor=4,
    sparsity_type="norm",
    mask_first_token=True
)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Some weights of the model checkpoint at ccdv/lsg-bart-large-4096 were not used when initializing LSGBartModel: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing LSGBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LSGBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
# from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
max_input = 512
max_target = 128
batch_size = 1
model_checkpoints = "ccdv/lsg-bart-large-4096"

In [ ]:
!pip install datasets
import datasets
from datasets import load_dataset, load_metric, load_from_disk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/NLP Project/Final Dataset 2/train.csv')
train_dataset = datasets.Dataset.from_pandas(df1)

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/NLP Project/Final Dataset 2/validation.csv')
validation_dataset = datasets.Dataset.from_pandas(df2)

In [ ]:
padding = "max_length"
ignore_pad_token_for_loss = True

In [ ]:
train_dataset

Dataset({
    features: ['Filename', 'judgement', 'summary'],
    num_rows: 799
})

In [ ]:
validation_dataset

Dataset({
    features: ['Filename', 'judgement', 'summary'],
    num_rows: 100
})

In [ ]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['judgement']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
train_tokenize_data = train_dataset.map(preprocess_data, batched = True, remove_columns=['Filename', 'judgement', 'summary'])


Map:   0%|          | 0/799 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
train_tokenize_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 799
})

In [ ]:
validation_tokenize_data = validation_dataset.map(preprocess_data, batched = True, remove_columns=['Filename', 'judgement', 'summary'])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
validation_tokenize_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [ ]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints, trust_remote_code = True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [ ]:
model.config

LSGBartConfig {
  "_name_or_path": "ccdv/lsg-bart-large-4096",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "adaptive": true,
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "LSGBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "auto_map": {
    "AutoModel": "modeling_lsg_bart.LSGBartModel",
    "AutoModelForCausalLM": "modeling_lsg_bart.LSGBartForCausalLM",
    "AutoModelForQuestionAnswering": "modeling_lsg_bart.LSGBartForQuestionAnswering",
    "AutoModelForSeq2SeqLM": "modeling_lsg_bart.LSGBartForConditionalGeneration",
    "AutoModelForSequenceClassification": "modeling_lsg_bart.LSGBartForSequenceClassification"
  },
  "base_model_prefix": "lsg",
  "block_size": 128,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout

In [ ]:
model.config.sparsity_type = "norm"

In [ ]:
batch_size = 2


In [ ]:
label_pad_token_id = -100

In [ ]:
model.config

LSGBartConfig {
  "_name_or_path": "ccdv/lsg-bart-large-4096",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "adaptive": true,
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "LSGBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "auto_map": {
    "AutoModel": "modeling_lsg_bart.LSGBartModel",
    "AutoModelForCausalLM": "modeling_lsg_bart.LSGBartForCausalLM",
    "AutoModelForQuestionAnswering": "modeling_lsg_bart.LSGBartForQuestionAnswering",
    "AutoModelForSeq2SeqLM": "modeling_lsg_bart.LSGBartForConditionalGeneration",
    "AutoModelForSequenceClassification": "modeling_lsg_bart.LSGBartForSequenceClassification"
  },
  "base_model_prefix": "lsg",
  "block_size": 128,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout

In [ ]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=label_pad_token_id,)

In [ ]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=False
    )

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=1cd195fcd8045eb95e18eac520c51d357ecf0dd9c056bd63c0ea29cfe941c97f
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [ ]:
metric = load_metric('rouge')

<ipython-input-34-10e4a614ebd6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
!pip install torchvision 
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_tokenize_data,
    eval_dataset=validation_tokenize_data,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,2.899567,12.042000,4.089100,10.101500,10.740200,20.000000
2,2.744300,2.821392,11.771100,4.736800,10.292700,11.005900,20.000000
2,2.268600,2.787759,11.617000,3.858800,9.777800,10.490200,20.000000
4,1.993500,2.769444,12.421700,4.143700,10.377300,11.142300,20.000000
4,1.993500,2.771739,12.430100,4.319500,10.426200,11.094700,20.000000


TrainOutput(global_step=1995, training_loss=2.2026951228168077, metrics={'train_runtime': 2024.3695, 'train_samples_per_second': 1.973, 'train_steps_per_second': 0.985, 'total_flos': 4323373681213440.0, 'train_loss': 2.2026951228168077, 'epoch': 4.99})

## REFERENCES

Inspired from: 

https://github.com/AldoF95/bart-chat-summarizer-finetuning/blob/main/Bart_large_xsum_fine_tuned_samsum.ipynb

https://huggingface.co/docs/transformers/model_doc/bart